In [1]:
import requests
import json
import sys

In [2]:
API_KEY_FILE = 'api.txt'


In [3]:
def readApiKey():
    try:
        with open(API_KEY_FILE, 'r') as f:
            for line in f:
                if line.strip().startswith('API_KEY='):
                    return line.strip().split('=', 1)[1]
        print(f"Error: API_KEY not found in '{API_KEY_FILE}'.")
        return None
    except FileNotFoundError:
        print(f"Critical Error: Configuration file '{API_KEY_FILE}'.")
        return None
    except Exception as e:
        print(f"Erro Reading API KeyL {e}.")
        return None

def readApiUrl():
    try:
        with open(API_KEY_FILE, 'r') as f:
            for line in f:
                if line.strip().startswith('API_URL='):
                    return line.strip().split('=', 1)[1]
        
        print(f"Error: 'API_URL=' not found in '{API_KEY_FILE}'.")
        return None
    except FileNotFoundError:
        print(f"Critical Error: Configuration file '{API_KEY_FILE}' not found.")
        return None
    except Exception as e:
        print(f"Error Reading API URL: {e}.")
        return None
    
def getFullUrlAndKey():
    apiKey = readApiKey()
    apiUrl = readApiUrl()
    if apiKey and apiUrl:
        return f"{apiUrl}?apikey={apiKey}"
    return None

In [4]:
def fetchLiveMatches():
    apiUrl = getFullUrlAndKey()
    if not apiUrl:
        print("Cannot fetch live matches due to missing API URL or Key.")
        return None
    try:
        response = requests.get(apiUrl, timeout=20)
        if not response.ok:
            print(" HTTP Error Encountered  ")
            print("API tool run failed and must not crash.")
            print(f"Error Status Code: {response.status_code}")
            print(f"Raw Response Text (for debugging):\n{response.text}")
            return None
        data = response.json()
        
        # api metadeta
        print("API Status Metadata:")
        info = data.get('info', {}) 
        for key, value in info.items():
            print(f"  {key}: {value}")
        print("-" * 50)
        
        #api stauses
        api_status = data.get('status')
        match_data = data.get('data')

        if api_status != "success":
            error_reason = data.get('reason', 'No specific reason provided.')
            print(f"API returned status: '{api_status}'. Error message:")
            print(f"Reason: {error_reason}")
            return

        if not match_data or not isinstance(match_data, list):
            print("API returned 'success', but the 'data' key is missing, empty, or invalid. No matches to report.")
            return
        #match details
        print(f"Live Match Report (Found {len(match_data)} matches):")
        
        for match in match_data:
            name = match.get('name', 'N/A')
            status = match.get('status', 'N/A')
            venue = match.get('venue', 'N/A')
            date = match.get('date', 'N/A')
            
            print("\n" + "=" * 50)
            print(f"a. Match Name:   {name}")
            print(f"b. Match Status: {status}")
            print(f"c. Venue:        {venue}")
            print(f"d. Date:         {date}")
            print("=" * 50)

            #score details
            scores = match.get('score')
            
            if scores and isinstance(scores, list) and len(scores) > 0:
                print("Score Details")
                
                for inning in scores:
                    inning_name = inning.get('inning', 'N/A')
                    runs = inning.get('r', 'N/A')
                    wickets = inning.get('w', 'N/A')
                    overs = inning.get('o', 'N/A')
                    
                    print(f"  Inning: **{inning_name}**")
                    print(f"    a. Innings (inning): {inning_name}")
                    print(f"    b. Runs (r):         {runs}")
                    print(f"    c. Wickets (w):      {wickets}")
                    print(f"    d. Overs (o):        {overs}")
                print("                  ")
            else:
                print("Score Details: Not available or match is complete.")
    
    #error handling
    except requests.exceptions.RequestException:
        print(" Network Error Encountered  ")
        print("API tool run failed and must not crash.")
        print("User-friendly error: Could not connect to the server. Check your internet connection or the API URL.")
    except Exception as e:
        print(" An Unexpected Error Occurred  ")
        print("API tool run failed and must not crash.")
        print(f"Unexpected error: {e}")
        
    print("\nReport generation complete.")


fetchLiveMatches()

API Status Metadata:
  hitsToday: 2
  hitsUsed: 1
  hitsLimit: 100
  credits: 0
  server: 14
  offsetRows: 0
  totalRows: 5
  queryTime: 492.3627
  s: 0
  cache: 1
--------------------------------------------------
Live Match Report (Found 5 matches):

a. Match Name:   Western Australia vs Queensland, 12th Match
b. Match Status: Day 2: Stumps - Western Australia trail by 180 runs
c. Venue:        W.A.C.A. Ground, Perth
d. Date:         2025-11-11
Score Details
  Inning: **Queensland Inning 1**
    a. Innings (inning): Queensland Inning 1
    b. Runs (r):         390
    c. Wickets (w):      10
    d. Overs (o):        105
  Inning: **Western Australia Inning 1**
    a. Innings (inning): Western Australia Inning 1
    b. Runs (r):         210
    c. Wickets (w):      5
    d. Overs (o):        74
                  

a. Match Name:   Northern Knights vs Otago, 15th Match
b. Match Status: No result (due to rain)
c. Venue:        Bay Oval, Mount Maunganui
d. Date:         2025-11-11
Score 